In [5]:
import os
from mp_api.client import MPRester
from pathlib import Path
from pymatgen.io.cif import CifWriter
import pandas as pd
ids= pd.read_csv('./data/material-data/mp-ids-46744.csv')
ids=list(pd.Series(ids[ids.columns[0]]))
import json
import ormsgpack as mp
def save_structure_to_cif(structure, output_file):
    with open(output_file, 'w') as f:
        cif_writer = CifWriter(structure)
        cif_writer.write_file(f.name)

def save_properties_to_json(properties:dict,output_file):
    with open(output_file, 'w') as f:
        json.dump(properties,f)

cifs_file='./data/root/data/cifs.bin'
def save_structures_to_bin(structures:dict, output_file=cifs_file):
    with open(output_file, 'wb') as f:
        f.write(mp.packb(structures, option=mp.OPT_NON_STR_KEYS))

props_file='./data/root/data/props.bin'
def save_properties_to_bin(properties:dict, output_file=props_file):
    with open(output_file, 'wb') as f:
        f.write(mp.packb(properties, option=mp.OPT_NON_STR_KEYS))

fields=['energy_per_atom','formation_energy_per_atom','band_gap','efermi','k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh','homogeneous_poisson']
    
def download_all():
    with MPRester("os.environ.get('MP_API_KEY')") as mpr:
        # show all fields
        # mpr.summary.search(material_ids=["mp-754118"])
    
        # print(mpr.summary.available_fields)
        # -> ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh', 'universal_anisotropy', 'homogeneous_poisson', 'e_total', 'e_ionic', 'e_electronic', 'n', 'e_ij_max', 'weighted_surface_energy_EV_PER_ANG2', 'weighted_surface_energy', 'weighted_work_function', 'surface_anisotropy', 'shape_factor', 'has_reconstructed', 'possible_species', 'has_props', 'theoretical', 'database_IDs']
        # print(d[0].structure)
        # docs = mpr.materials.summary.search(material_ids=["mp-10114"], fields=['material_id',"structure", 'efermi'])
        # print(docs[0].efermi)
        # print(docs[0].model_dump(include=['efermi']))
    
        docs=mpr.materials.summary.search(material_ids=ids, fields=fields+['structure','material_id'])
    
    
    props=dict()
    cifs=dict()
    for d in docs:
        output_file = f"./data/root/{d.material_id}.cif"
        # save_structure_to_cif(d.structure, output_file)
        # save_properties_to_json(d.model_dump(include=fields),f"props/{d.material_id}.json")
        cifs[d.material_id]=d.structure.as_dict()
        props[d.material_id]=d.model_dump(include=fields)
    save_structures_to_bin(cifs)
    save_properties_to_bin(props)
        # uncorrected_energy_per_atom energy_per_atom formation_energy_per_atom band_gap efermi 
        #  see https://docs.materialsproject.org/methodology/materials-methodology/elasticity/ k <-> bulk, g <-> shear
        # k_vrh g_vrh homogeneous_poisson
# download_all()

# with zstd
import zstandard as zstd
def compress_all():
    with open(cifs_file,'rb') as f:
        res=zstd.compress(f.read(), level=10)
    with open('./data/root/cifs.zstd','wb') as f:
        f.write(res)
    with open(props_file,'rb') as f:
        res=zstd.compress(f.read(),level=10)
    with open('./data/root/props.zstd','wb') as f:
        f.write(res)
# compress_all()

def uncompress_all():
    with open('./data/root/cifs.zstd','rb') as f:
        res=zstd.decompress(f.read())
    with open(cifs_file,'wb') as f:
        f.write(res)
    with open('./data/root/props.zstd','rb') as f:
        res=zstd.decompress(f.read())
    with open(props_file,'wb') as f:
        f.write(res)
uncompress_all()

In [16]:
# print data statistics (Table 1)
# basedir = './data/root/props/'
basedir = './data/root/data/'
# propfiles = os.listdir(basedir) 
def load_properties_from_json(file):
    filename = os.path.basename(file)
    with open(basedir+file, 'rb') as f:
        properties=json.load(f)
        return (filename, properties)
    return None
def load_properties_from_bin(file):
    filename = os.path.basename(file)
    with open(file, 'rb') as f:
        properties=mp.unpackb(f.read())
        return properties
    return None
# propdata = dict()
# for file in propfiles:
#     name,prop=load_properties_from_json(file) 
#     for key in prop.keys():
#         propdata.setdefault(key,[])
#         propdata[key].append(prop[key])

f=pd.DataFrame(load_properties_from_bin(basedir+'props.bin')).transpose()
import numpy as np
for moduli in ['k_voigt','k_reuss',	'k_vrh', 'g_voigt',	'g_reuss', 'g_vrh']:
    f[moduli]=np.log(f[moduli])
f.describe()

/home/nodoteve/apps/pythom/miniconda/envs/cgcnn2/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,energy_per_atom,formation_energy_per_atom,band_gap,efermi,k_voigt,k_reuss,k_vrh,g_voigt,g_reuss,g_vrh,homogeneous_poisson
count,36419.000000,36419.000000,36419.000000,36419.000000,5135.000000,5124.000000,5135.000000,5100.000000,4962.000000,5056.000000,5145.000000
mean,-6.202638,-1.557713,1.401998,3.176433,4.442641,4.365307,4.412232,3.731682,3.540498,3.640171,0.290299
std,1.877744,1.116585,1.753288,2.814591,0.832734,0.955244,0.865331,0.875033,1.057755,0.934253,0.267467
min,-14.076122,-4.499302,0.000000,-14.017281,-0.058679,-1.673020,-0.117437,-1.944802,-3.794388,-1.412757,-13.047463
25%,-7.561379,-2.458886,0.000000,1.169531,3.951460,3.890540,3.914757,3.234844,2.986730,3.105451,0.250164
50%,-6.314644,-1.499015,0.457600,3.021116,4.633773,4.606371,4.620210,3.859316,3.711011,3.773314,0.291422
75%,-4.850119,-0.569807,2.560700,5.132208,5.074705,5.064164,5.068715,4.360637,4.291922,4.327258,0.331566
max,-0.005272,4.309211,16.586400,17.758767,6.687777,6.639926,6.664137,6.055889,7.600026,6.912284,5.604522


In [17]:
# prepare for train
def set_property_to_ids(property):
    f[property].to_csv('./data/root/data/id_prop.csv',index=True,header=False)
set_property_to_ids('energy_per_atom')